<div style="font-size: 22px; line-height: 1.6;">

# Marine Carbonate Visualizer - Bjerrum Plot

<div style="text-align: center; margin: 30px 0;">
    <img src="figures/carbonate_system.svg" width="800" alt="Marine Carbonate System Diagram" style="border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); max-width: 100%;">
</div>

<div style="text-align: center; font-size: 18px; color: #00204C; margin: 20px 0;">
    <strong>Interactive Bjerrum diagram for carbonate system analysis</strong>
</div>

</div>

<div style="font-size: 22px; line-height: 1.6;">

---

## Interactive Bjerrum Plot for Carbonate Systems

This tool provides **Bjerrum plot visualization** showing how changes in **Total Alkalinity (TA)** and **Dissolved Inorganic Carbon (DIC)** affect the marine carbonate system speciation, using the Python package **PyCO2SYS**.

---

#### What does the Bjerrum plot show?

**Input parameters:**
* **Total alkalinity** (TA) in umol/kg
* **Dissolved Inorganic Carbon** (DIC) in umol/kg

**Fixed conditions:**
* **Salinity** = 35 PSU
* **Temperature** = 25 degrees C

**Interactive visualizations:**
1. **Bjerrum Plot** - Species fractions (alpha) vs pH showing CO2, HCO3-, and CO3-2 distribution
2. **Current Composition** - Pie chart of species at current pH
3. **pH-pCO2 Relationship** - How pH affects partial pressure of CO2
4. **Complete Analysis** - Detailed numerical results and system parameters

**Educational Value:**
- Visualize chemical equilibria in seawater
- Understand buffer capacity and pH control
- Explore ocean acidification effects
- Analyze calcification potential

</div>

In [ ]:
import PyCO2SYS as pyco2

def compute_carbonate_system(alkalinity, DIC, salinity=35, temperature=25):
    # Run CO2SYS using alkalinity and DIC as input parameters
    results = pyco2.sys(
        par1=alkalinity,      # par1_type = 1 - Total Alkalinity (umol/kg)
        par2=DIC,             # par2_type = 2 - DIC = Dissolved Inorganic Carbon (umol/kg)
        par1_type=1,          # 1 = total alkalinity
        par2_type=2,          # 2 = DIC
        salinity=salinity,    # Salinity in PSU
        temperature=temperature  # Temperature in degrees C
    )

    # Return selected key results in a dictionary
    return {
        "pH_total": results["pH_total"],         # pH on the total scale
        "pCO2": results["pCO2"],                 # partial pressure of CO2 (uatm)
        "bicarbonate": results["bicarbonate"],   # HCO3- concentration (umol/kg)
        "carbonate": results["carbonate"],       # CO3-2 concentration (umol/kg)
        "omega_aragonite": results["saturation_aragonite"],  # saturation state with respect to aragonite
        "salinity": salinity,
        "temperature": temperature
    }



In [ ]:
from IPython.display import display, HTML

def show_conditions(data):
    html = f"""
    <div style="font-size: 20px; color: #00204C; font-family: sans-serif; margin-top: 15px;">
        <p><strong>Conditions used in the calculation:</strong></p>
        <p>Salinity: {data['salinity']:.0f} PSU</p>
        <p>Temperature: {data['temperature']:.0f} degrees C</p>
    </div>
    """
    display(HTML(html))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def create_bjerrum_plot(data):
    """Create an optimized Bjerrum plot - no duplicates"""
    
    # Close any existing figures to prevent duplicates
    plt.close('all')
    
    # Create figure with smaller size for faster rendering
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Marine Carbonate System - Bjerrum Plot', fontsize=18, fontweight='bold')
    
    # Use simple colors instead of viridis colormap for speed
    colors = ['#440154', '#31688e', '#35b779', '#fde725', '#21908c']
    
    # Reduced pH range for faster calculation (50 points instead of 100)
    pH_range = np.linspace(4, 11, 50)
    
    # Pre-calculate constants once
    K1 = 1e-6   # First dissociation constant 
    K2 = 1e-9.3 # Second dissociation constant 
    
    # Vectorized calculation for speed
    H = 10**(-pH_range)
    denominator = H**2 + K1*H + K1*K2
    
    alpha0 = H**2 / denominator  # CO2
    alpha1 = K1*H / denominator  # HCO3-
    alpha2 = K1*K2 / denominator # CO3-2
    
    # Plot 1: Bjerrum Plot - simplified styling
    ax1.plot(pH_range, alpha0, color=colors[0], linewidth=3, label='CO2')
    ax1.plot(pH_range, alpha1, color=colors[1], linewidth=3, label='HCO3-')
    ax1.plot(pH_range, alpha2, color=colors[2], linewidth=3, label='CO3-2')
    
    # Mark current pH - simplified
    current_pH = data['pH_total']
    ax1.axvline(x=current_pH, color='red', linestyle='--', linewidth=2)
    ax1.text(current_pH + 0.1, 0.8, f'pH: {current_pH:.2f}', 
             fontsize=11, color='red', fontweight='bold')
    
    ax1.set_xlabel('pH', fontsize=14)
    ax1.set_ylabel('Fraction', fontsize=14)
    ax1.set_title('Bjerrum Plot', fontsize=16, fontweight='bold')
    ax1.legend(fontsize=12)
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(4, 11)
    ax1.set_ylim(0, 1)
    
    # Plot 2: Simple pie chart
    current_co2 = data["pCO2"] * 0.034
    species_values = [current_co2, data["bicarbonate"], data["carbonate"]]
    species_labels = ['CO2', 'HCO3-', 'CO3-2']
    
    ax2.pie(species_values, labels=species_labels, autopct='%1.1f%%', 
            colors=colors[:3], startangle=90)
    ax2.set_title(f'Composition at pH {current_pH:.2f}', fontsize=14, fontweight='bold')
    
    # Plot 3: Saturation state - simplified
    omega = data["omega_aragonite"]
    ax3.bar(['Aragonite'], [omega], color=colors[3] if omega >= 1 else colors[0])
    ax3.axhline(y=1, color='black', linestyle='--', alpha=0.7)
    ax3.set_ylabel('Omega', fontsize=14)
    ax3.set_title('Saturation State', fontsize=14, fontweight='bold')
    ax3.text(0, omega*0.5, f'Ω = {omega:.2f}', ha='center', fontsize=12, fontweight='bold')
    
    # Plot 4: Simplified system info
    ax4.axis('off')
    
    # Shorter text for faster rendering
    info_text = f"""Input:
TA: {(data['bicarbonate'] + 2*data['carbonate']):.0f} μmol/kg
DIC: {data['bicarbonate'] + data['carbonate'] + current_co2:.0f} μmol/kg

Results:
pH: {data['pH_total']:.2f}
pCO2: {data['pCO2']:.0f} μatm
[HCO3-]: {data['bicarbonate']:.0f} μmol/kg
[CO3-2]: {data['carbonate']:.0f} μmol/kg
Ω_arag: {data['omega_aragonite']:.2f}

Conditions: S={data['salinity']}, T={data['temperature']}°C"""
    
    ax4.text(0.05, 0.95, info_text, transform=ax4.transAxes, fontsize=11,
             verticalalignment='top', fontfamily='monospace')
    
    plt.tight_layout()
    plt.show()
    
    # Return figure and close it after showing to prevent duplicates
    return fig

In [ ]:
from ipywidgets import FloatSlider, Layout, HTML, Button
from IPython.display import display

# Optimized slider styling - simpler for faster rendering
slider_style = {'description_width': '200px'}
slider_layout = Layout(width='700px', margin='10px 0')

# Alkalinity slider - optimized
alk_slider = FloatSlider(
    value=2300, min=1800, max=3500, step=50,  # Larger step for faster updates
    description="Alkalinity (μmol/kg):",
    style=slider_style, 
    layout=slider_layout,
    continuous_update=False,  # Only update on release for speed
    readout=True, 
    readout_format=".0f"
)

# DIC slider - optimized  
dic_slider = FloatSlider(
    value=2020, min=1600, max=3500, step=50,  # Larger step for faster updates
    description="DIC (μmol/kg):",
    style=slider_style, 
    layout=slider_layout,
    continuous_update=False,  # Only update on release for speed
    readout=True, 
    readout_format=".0f"
)

# Simplified CSS for faster loading
custom_css = HTML("""
<style>
.widget-label {
    font-size: 16pt !important;
    font-weight: bold;
    color: #003366 !important;
}
.widget-readout {
    font-size: 16pt !important;
    font-weight: bold;
    color: #003366 !important;
}
</style>
""")

print("✅ Optimized sliders loaded successfully!")
print("📊 Use larger steps (50 μmol/kg) for faster response")
print("🔄 Plots update only when you release the slider")

In [ ]:
from ipywidgets import Output, VBox, interactive_output, Button, Layout
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# Set matplotlib backend for faster rendering and avoid duplicates
plt.ioff()  # Turn off interactive mode
plt.rcParams['figure.max_open_warning'] = 0  # Avoid warnings about too many figures

# Create output widget
output = Output()

# Optimized update function to prevent duplicates
def update_display(alkalinity, DIC):
    """Fast update function with duplicate prevention"""
    try:
        # Close any existing figures to prevent memory leaks and duplicates
        plt.close('all')
        
        # Compute carbonate system
        data = compute_carbonate_system(alkalinity, DIC)
        
        # Clear and update output efficiently
        with output:
            clear_output(wait=True)
            create_bjerrum_plot(data)
            
    except Exception as e:
        with output:
            clear_output(wait=True)
            print(f"Error: {e}")
            print("Please check your input values and try again.")

# Fast reset function
def reset_values(button):
    """Quick reset to default values"""
    alk_slider.value = 2300
    dic_slider.value = 2020

# Create reset button with faster styling
reset_button = Button(
    description="Reset Values",
    button_style='info',
    layout=Layout(width='150px', margin='5px 0')
)
reset_button.on_click(reset_values)

# Create interactive output - optimized for speed and no duplicates
interactive_plot = interactive_output(update_display, {
    'alkalinity': alk_slider, 
    'DIC': dic_slider
})

# Display custom CSS - simplified for speed
display(custom_css)

# Simplified interface layout
interface = VBox([
    alk_slider,
    dic_slider,
    reset_button,
    interactive_plot  # Use interactive_plot instead of output to avoid duplicates
])

# Display interface
display(interface)

print("✅ Interface loaded - no duplicate plots!")

**Author:** Cardoso-Mohedano JG 
**Institution:** Instituto de Ciencias del Mar y Limnología, UNAM, Estación El Carmen  
**License:** [CC BY-NC 4.0](https://creativecommons.org/licenses/by-nc/4.0/)  
**ORCID:** [0000-0002-2918-972X](https://orcid.org/0000-0002-2918-972X)

---
This interactive tool allows exploration of the marine carbonate system using [PyCO2SYS](https://pyco2sys.readthedocs.io/en/latest/).  
Developed with support from [Claude AI](https://claude.ai) by Anthropic and [OpenAI ChatGPT](https://openai.com/chatgpt) and educational Python tools.
